In [1]:
import networkx as nx
import gurobipy as gb
from itertools import combinations, chain
#import pygraphviz as pygv
import os
from IPython.display import SVG, display

In [2]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

#
# Drawing 
# functions
#def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

#
# Drawing 
# functions
#

def DrawInitialGraph():
    global DrawG
    DrawG = pygv.AGraph(undirected='true', strict='true', splines='true')


    for i in G.nodes():
        pos = str(G.node[i]['x'] * args.scale) + ',' + str((G.node[i]['y'])* args.scale)
        DrawG.add_node (i, shape='circle', pos=pos, width=0.1, fontsize=9)
    
    DrawG.layout(prog='neato', args='-n')
    DrawG.draw (path=str(basename) + '.svg', format='svg')
    

def DrawSol (x):
    for i,j in G.edges():
        lab = x[i,j].x
        G[i][j]['capacity'] = x[i,j].x
        if x[i,j].x > 0.00001:
            if x[i,j].x > 0.999999:
                DrawG.add_edge(i, j, color='black', label=lab, fontsize=9)
            else:
                DrawG.add_edge(i, j, color='red', label=lab)
    path=str(basename)+'_sol.svg'
    DrawG.draw (path=path, format='svg')
    DrawG.delete_edges_from(DrawG.edges())

def DrawInitialGraph():
    global DrawG
    DrawG = pygv.AGraph(undirected='true', strict='true', splines='true')


    for i in G.nodes():
        pos = str(G.node[i]['x'] * args.scale) + ',' + str((G.node[i]['y'])* args.scale)
        DrawG.add_node (i, shape='circle', pos=pos, width=0.1, fontsize=9)
    
    DrawG.layout(prog='neato', args='-n')
    DrawG.draw (path=str(basename) + '.svg', format='svg')
    

def DrawSol (x):
    for i,j in G.edges():
        lab = x[i,j].x
        G[i][j]['capacity'] = x[i,j].x
        if x[i,j].x > 0.00001:
            if x[i,j].x > 0.999999:
                DrawG.add_edge(i, j, color='black', label=lab, fontsize=9)
            else:
                DrawG.add_edge(i, j, color='red', label=lab)
    path=str(basename)+'_sol.svg'
    DrawG.draw (path=path, format='svg')
    DrawG.delete_edges_from(DrawG.edges())

In [3]:
class args:
    filename = None
    scale = 40

## An integer ﬁxed-charge multicommodity ﬂow (FCMF) model for train unit scheduling

$$
y_a = \begin{cases}
1 \text{ if arc $a$ is in the tree} \\
0 \text { otherwise}
\end{cases}
$$


\begin{alignat}{3}
\min W_{1}\sum_{k \in K} \sum_{p \in P^k}c_{p}x_{p} + W_{2}\sum_{a \in A}y_{a} &&\\
\sum_{p \in P^k}x_{p} \le b^k &&\;\; \forall k \in K\\
\sum_{k \in K_{j}} \sum_{p \in P^k_{j}} D^j_{f,k}x_{p} \le d^j_{f} &&\;\; \forall f \in \bar{F}_{j} \text{ , } \forall j \in N\\
\sum_{k \in K_{a}} \sum_{p \in P^k_{a}} x_{p} \le m_{a}y_{a} &&\;\; \forall a \in A\\
\sum_{a \in \delta^-(j)} y_{a} = 1 &&\;\; \forall j \in N^-_{B}\\
\sum_{a \in \delta^+(j)} y_{a} = 1 &&\;\; \forall j \in N^+_{B}\\
\tau^D_{arr(i)}(\sum_{a \in \delta^+(i)} y_{a} - 1) + \tau^C_{dep(j)}(\sum_{a \in \delta^-(j)} y_{a} - 1) \le e_{ij} && \;\; \forall (i,j) \in A^*\\
x_{p} \in Z_{+}, \; \forall p \in P^k, \; \forall k \in K; \; y_{a} \in \{0,1\}, \; \forall a \in A\\
\end{alignat}

### Variables are declared BINARY but the polytope is integral

In [4]:
args.filename = "mst.gml"
args.scale = 40

basename = os.path.splitext(args.filename)[0]
#G = nx.read_graphml (args.filename, node_type=int)
G = nx.complete_graph(3)
print ("G has", G.number_of_nodes(), "nodes and", G.number_of_edges(), "edges")

G has 3 nodes and 3 edges


In [5]:
D = G.to_directed()
root = 0
#DrawInitialGraph()
#display(SVG(filename=basename+'.svg'))

In [6]:
# adding s and t as nodes
nodessetplusst = list(G.nodes())
nodessetplusst.append(4)
nodessetplusst.append(5)
nodessetplusst

[0, 1, 2, 4, 5]

In [7]:
commodities = [x for x in nodessetplusst if x != 4]
commodities

[0, 1, 2, 5]

In [8]:
# addind some arcs outgoing from s and ingoing to t
arcssetplusst = list(G.edges())
for i in range(0,4):
    arcssetplusst.append((4,i))
    arcssetplusst.append((i,5))
arcssetplusst

[(0, 1),
 (0, 2),
 (1, 2),
 (4, 0),
 (0, 5),
 (4, 1),
 (1, 5),
 (4, 2),
 (2, 5),
 (4, 3),
 (3, 5)]

In [9]:
# also we use a path variable x as a G[u][v]['cost'] for u,v in D.edges() result, because we don't have number of units in our graph

In [10]:
# all unit types - have to be added to a structure of the graph like an extra value
K = 3

# have to calculate w1 and w2 for an objective function
w1 = 3
w2 = 2

In [11]:
train_cars = {0:3, 1:3, 2:2}
train_cost = {0:15, 1:20, 2:50}
train_capacity = {0:219, 1:230, 2:145}

In [12]:
fcmf = gb.Model()


x = fcmf.addVars(D.edges(), \
                        obj=[w1 * train_cost.get((u)) for u,v in D.edges()],\
                        vtype=gb.GRB.INTEGER,\
                        name='x')

y = fcmf.addVars(D.edges(), ub=1.0, \
                         obj = w2, \
                         vtype=gb.GRB.BINARY, name='y')

f = fcmf.addVars(D.edges(), commodities, \
                         ub=G.number_of_nodes(), \
                         vtype=gb.GRB.CONTINUOUS, name='f')

fcmf.update()
fcmf.write('fcmf4.lp')

In [13]:
#
# Constraints with fleet size upper bound
#

# b - fleet size upper bound
b = 2000

for u,v in D.edges():
    fcmf.addConstr(train_cost.get((u)) <= b)

fcmf.update()
fcmf.write('fcmf4.lp')

In [14]:
#
# Coupling constraints
#

# m = maximim number of coupled units flowing through arc
# have to be added to a structure of the graph like an extra value
m = 10

for u,v in D.edges():
    for x, y in G.edges():
        fcmf.addConstr(train_cost.get((u)) <= m * train_cost.get((u)))
            

fcmf.update()
fcmf.write('fcmf4.lp')

In [15]:
#
# To forbid coupling / decoupling in banned locations
#

for v, k in D.edges():
    if (v != k):
        fcmf.addConstr(f.sum('*',v,k) == 1)
        fcmf.addConstr(f.sum(v,'*',k) == 1)

fcmf.update()
fcmf.write('fcmf4.lp')

### Time for a single decoupling/coupling operation at the arrival/departure location of train i/j
\begin{alignat}{3}
\tau^D_{arr(i)}\\
\tau^C_{dep(j)}\\
\end{alignat}

In [16]:
#
# To ensure time allowance validity for coupling/decoupling 
#

# lets say time for operation is equal to 1
# slack time between i and j is 2

for v, k in D.edges():
    if (v != k):
        fcmf.addConstr((f.sum(v,'*',k) - 1) + (f.sum('*',v,k) - 1) <= 2)

fcmf.update()
fcmf.write('fcmf4.lp')

In [17]:
#
# Check number of coupled cars and coupling upper bound for combination
#

ub = 6

for v, k in D.edges():
    if (v != k):
        fcmf.addConstr(train_cars.get(v) + train_cars.get(k) <= ub)

fcmf.update()
fcmf.write('fcmf4.lp')

In [18]:
#
# Check passenger capacity
#

cap = 23

fcmf.addConstrs((train_capacity.get((i)) >= cap\
                       for (i,j) in D.edges()), name='Cap')

fcmf.update()
fcmf.write('fcmf4.lp')

In [19]:
fcmf.optimize()

Optimize a model with 54 rows, 36 columns and 48 nonzeros
Variable types: 24 continuous, 12 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [20]:
#DrawSol(x)
#display(SVG(filename=basename+'_sol.svg'))